# BMI/CS 576 - HW6 - Fall 2023
The objectives of this homework are to better understand

* Gaussian mixture model-based clustering
* Bottom-up hierarchical clustering
* scoring Bayesian networks with model evidence
* determining causal relationships

## HW policies
Before starting this homework, please read over the [homework policies](https://canvas.wisc.edu/courses/374201/modules/items/6356820) for this course.  In particular, note that homeworks are to be completed *individually* and plagiarism from any source (with the one exception noted below) will be considered **academic misconduct**.

You are welcome to use any code from the weekly notebooks (including the official solutions) in your solutions to the HW.

## PROBLEM 1: Gaussian mixture model-based clustering (25 points)

Run the EM algorithm (either by hand or with your **own** code) for Gaussian mixture model-based
clustering on the the following set of one-dimensional points: $X = (x_1,x_2,x_3,x_4,x_5) = (-5,-2,0,3,9)$ for **two** iterations.
Let $k=2$, the initial cluster means be $\mu_1 = -4$ and $\mu_2 = 3$,
the initial cluster prior probabilities be $P_1 = P_2 = 0.5$, and the
variances be $\sigma^2_1 = \sigma^2_2 = 9$.  You should treat the variances as fixed
parameters that are not updated during EM.  After each iteration, show 

**(i)** the probabilities of each point being assigned to each cluster

**(ii)** the cluster that is most probable for each point

**(iii)** the updated cluster means, and 

**(iv)** the updated cluster prior probabilities. 

*If you run the algorithm via code, you must present the output in a nicely formatted manner.*

### BEGIN SOLUTION TEMPLATE=solution to problem 1
![p1_solution](em_solution.png)

### END SOLUTION

## PROBLEM 2: Bottom-up hierarchical clustering (25 points)

Suppose we wish to compute a hierarchical clustering of the following one-dimensional points $(A, B, C, D, E) = (0, 6, 11, 15, 18)$.  Below is the Euclidean distance matrix for these points.

|       | A | B  |  C | D  | E  |
|-------|---|----|----|----|----|
| **A** | 0 | 6  | 11 | 15 | 18 |
| **B** | 6 | 0  |  5 | 9  | 12 |
| **C** | 11| 5  |  0 | 4  | 7  |
| **D** | 15| 9  |  4 | 0  | 3  |
| **E** |18 | 12 |  7 | 3  | 0  |

**(a)** Compute a hierarchical clustering of these points using *single link*.  Give the intermediate tree and distance matrix after each iteration.

**(b)** Compute a hierarchical clustering of these points using *complete link*.  Give the intermediate tree and distance matrix after each iteration.

**(b)** Compute a hierarchical clustering of these points using *average link*.  Give the intermediate tree and distance matrix after each iteration.

### BEGIN SOLUTION TEMPLATE=solution to problem 2

![p2_solution](p2_solution.png)

### END SOLUTION

## PROBLEM 3 (20 POINTS)

Consider the Bayesian network shown below

![p3 network](p3_network.png)

The model evidence formula for this network is

$$\log P(D|G) = -\left(\log(n+1) + \log {n \choose n_{1,0}} + \log(n+1) + \log {n \choose n_{3,0}} + \sum_{a \in \{0, 1\}} \left(\log (n_{1,a} + 1) + \log {n_{1,a} \choose n_{2,a,0} } \right)  \right)$$

where the data dependent values (the "sufficient statistics") are defined as follows:

$$n_{1,a} = |\{i: x_{1,i} = a\}|$$
$$n_{3,a} = |\{i: x_{3,i} = a\}|$$
$$n_{2,a,c} = |\{i: x_{1,i} = a, x_{2,i} = c\}|$$
$$n = {|\{i\}|}$$

In this problem we will use the same dataset as in notebook 24 (provided as the file [`data.txt`](data.txt) and read in with the cell below).

**(a)** For this dataset, compute the values of the sufficient statistics (defined above) for the model evidence of this network.  Show your code for computing these values.

**(b)** Given your values from (a), compute the model evidence (defined above) of this network.  Show your work.


**(c)** Compare your answer from (b) to the model evidence computed for the network in notebook 24.  Which network structure is a better fit for this dataset?  Justify your answer.


In [ ]:
# read in the data set as a list of tuples 
# (each tuple is one joint observation of the three variables)
data = [tuple(map(int, line.split())) for line in open("data.txt")]

### BEGIN SOLUTION TEMPLATE=solution to problem 3

**(a)** Computed with the code below.
$$\begin{eqnarray}
n & = & 1000 \\
n_{1,0} & = & 733 \\
n_{1,1} & = & 267 \\
n_{3,0} & = & 734 \\
n_{2,0,0} & = & 542 \\
n_{2,1,0} & = & 196
\end{eqnarray}
$$

**(b)** Computed with the code below
$$\log P(D|G) \approx -1747$$

**(c)** The model evidence of this network is less than that from notebook 24 (which was $\approx -1579$), and thus  the network in this problem is a worse fit for the given dataset.

### END SOLUTION

In [ ]:
### BEGIN SOLUTION TEMPLATE=any code used for your solution to problem 3
import bayesian_network
import itertools
import pprint
import math

def logbinom(n, k):
    """The natural logarithm of the binomial coefficient (n choose k)"""
    return math.lgamma(n + 1) - math.lgamma(k + 1) - math.lgamma(n - k + 1)

def sufficient_statistics(bn, data):
    ss = []
    for i in range(bn.num_vertices()):
        parent_possible_values = [bn.possible_values[j] for j in bn.parents(i)]
        vertex_ss = {parent_vals: [0] * len(bn.possible_values[i])
                     for parent_vals in itertools.product(*parent_possible_values)}
        ss.append(vertex_ss)

    for values in data:
        encoded_values = bn.encode_values(values)
        for i, value in enumerate(encoded_values):
            parent_values = tuple(encoded_values[j] for j in bn.parents(i))
            ss[i][parent_values][value] += 1

    return ss

def model_evidence(bn, data):
    """assumes binary random variables"""
    ss = sufficient_statistics(bn, data)
    me = 0
    for i, vertex_ss in enumerate(ss):
        for count_vector in vertex_ss.values():
            total_counts = sum(count_vector)
            me -= (math.log(total_counts + 1) + logbinom(total_counts, count_vector[0]))
    return me

random_variables = ["x1", "x2", "x3"]
g = bayesian_network.BayesianNetwork(random_variables)
g.set_cpd("x1",
          [], [0, 1],
          {(): [0.75, 0.25]})
g.set_cpd("x2",
          ["x1"], [0, 1],
          {(0,): [0.9, 0.1],
           (1,): [0.3, 0.7]})
g.set_cpd("x3",
          [], [0, 1],
          {(): [0.75, 0.25]})

log_prob_data_given_graph = model_evidence(g, data)
pprint.pprint(sufficient_statistics(g, data), width=20)
print("n =", len(data))
print(log_prob_data_given_graph)

### END SOLUTION

## PROBLEM 4 (30 POINTS)

Suppose we wish to construct a Bayesian network representing the gene regulatory network for two genes, $X$ and $Y$.
We are given data from 80 independent observational experiments in which the expression levels of the two genes are measured.  The expression level of each gene is measured as either "low" (L) "medium" (M), or "high" (H).  Below is a table summarizing the number of times (count) each configuration of gene expression status was observed in these experiments.


| X | Y | count |
|---|---|-------|
| L | L |   4   |
| L | M |   8   |
| L | H |  16   |
| M | L |   4   |
| M | M |   6   |
| M | H |   2   |
| H | L |  32   |
| H | M |   6   |
| H | H |   2   |

**(a)** Give a table specifying the joint probability distribution, $P(X, Y)$, that we would estimate from these data.

**(b)** Compute the mutual information (use the natural log) between $X$ and $Y$ given your table from (a).  Show your work.  Assuming the estimated distribution in (a) is the true distribution, are $X$ and $Y$ independent of each other?  Justify your answer.

**(c)** Suppose we use the Bayesian network structure below.  Give the CPDs for $X$ and $Y$ such that the joint distribution represented by the network is the same as that estimated in (a).

![p4 network c](p4_network_c.png)

**(d)** Suppose we use the alternative Bayesian network structure below.  Give the CPDs for $X$ and $Y$ such that the joint distribution represented by the network is the same as that estimated in (a).

![p4 network d](p4_network_d.png)

**(e)** You are now given data from an intervention experiment in which gene $Y$ is forced to be "low" through an intervention (e.g., with a [small interfering RNA](https://en.wikipedia.org/wiki/Small_interfering_RNA) targeting $Y$).  This experiment is repeated 100 times, giving the following counts for the measurements of gene $X$.

| X | count |
|---|-------|
| L |  10   |
| M |  10   |
| H |  80   |

Which of the two networks considered in (c) and (d) is more likely to better model the true causal relationships in the gene regulatory network?  Justify your answer.


### BEGIN SOLUTION TEMPLATE=solution to problem 4

**(a)** 

| X | Y | P(X, Y) |
|---|---|-------|
| L | L | 0.05 |
| L | M | 0.10 |
| L | H | 0.20 |
| M | L | 0.05 |
| M | M | 0.075 |
| M | H | 0.025 |
| H | L | 0.40 |
| H | M | 0.075 |
| H | H | 0.025 |

**(b)** 

| X | Y | P(X, Y) | P(X) | P(Y) | P(X,Y) log(P(X,Y)/(P(X)P(Y))) |
|---|---|---------|------|------|-------------------------------|
| L | L | 0.05 | 0.35 | 0.50 | -0.0626 |
| L | M | 0.10 | 0.35 | 0.25 | +0.0134 |
| L | H | 0.20 | 0.35 | 0.25 | +0.1650 |
| M | L | 0.05 | 0.15 | 0.50 | -0.0203 |
| M | M | 0.075 | 0.15 | 0.25 | +0.0520 |
| M | H | 0.025 | 0.15 | 0.25 | -0.0101 |
| H | L | 0.40 | 0.50 | 0.50 | +0.1880 |
| H | M | 0.075 | 0.50 | 0.25 | -0.0383 |
| H | H | 0.025 | 0.50 | 0.25 | -0.0402 |


$$I(X,Y) \approx 0.247$$

(with log base 2, $I(X,Y)=0.356$, and with log base 10, $I(X,Y)=0.107$ )


**(c)** 

P(X)

| X | P(X) |
|---|------|
| L | 0.35 |
| M | 0.15 |
| H | 0.50 |

P(Y|X)

| X | P(Y=L\|X) | P(Y=M\|X) | P(Y=H\|X) |
|---|----------|----------|----------|
| L | 0.143 | 0.286 | 0.571 | 
| M | 0.333 | 0.50 | 0.167 | 
| H | 0.80 | 0.15 | 0.05 | 


**(d)** 

P(Y)

| Y | P(Y) |
|---|------|
| L | 0.50 |
| M | 0.25 |
| H | 0.25 |

P(X|Y)

| Y | P(X=L\|Y) | P(X=M\|Y) | P(X=H\|Y) |
|---|----------|----------|----------|
| L | 0.10 | 0.10 | 0.80 | 
| M | 0.40 | 0.30 | 0.30 | 
| H | 0.80 | 0.10 | 0.10 | 

**(e)** The distribution of X in this intervention data is 

| X |     P(X)       |
|---|----------------|
| L |  10/100 = 0.1  |
| M |  10/100 = 0.1  |
| H |  80/100 = 0.8  |

This distribution is markedly different from the marginal distribution of X in the observational data and identical to the conditional distribution $P(X | Y = L)$ found in part (d).  This suggests that Y is causal of X and that the Bayesian network from part (d) is the better causal model.  Had the opposite been the case (X causal of Y), we would have expected this intervention distribution of X to be similar to that in the observational data.


### END SOLUTION